In [3]:
import math
import random
from typing import List, Dict
from sentence_transformers import SentenceTransformer, util

random.seed(42)


e:\CODING\DevApp\mentorme\.venv2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
tutors: List[Dict] = [
    {
        "id": 1,
        "name": "Nguyễn Văn A",
        "subjects": ["Toán"],
        "grades": [8, 9],
        "lat": 21.01,
        "lng": 105.80,
        "hourly_rate": 150_000,
        "profile_completed": 0.9,
        "is_verified_id": True,
        "is_verified_student_card": True,
        "total_classes": 12,
        "completed_classes": 12,
        "avg_rating": 4.8,
        "avg_response_time_sec": 300,
        "policy_violations_count": 0,
        "free_slots": ["Mon_19-21", "Wed_19-21"],
        "bio": "Gia sư Toán 9, luyện thi vào 10, thế mạnh hình học và tư duy giải nhanh",
    },
    {
        "id": 2,
        "name": "Trần Thị B",
        "subjects": ["Toán"],
        "grades": [6, 7, 8, 9],
        "lat": 21.03,
        "lng": 105.81,
        "hourly_rate": 130_000,
        "profile_completed": 0.8,
        "is_verified_id": True,
        "is_verified_student_card": False,
        "total_classes": 7,
        "completed_classes": 6,
        "avg_rating": 4.3,
        "avg_response_time_sec": 1800,
        "policy_violations_count": 0,
        "free_slots": ["Mon_18-20", "Wed_19-21"],
        "bio": "Dạy Toán cơ bản đến nâng cao, ôn nền tảng cho học sinh mất gốc",
    },
    {
        "id": 3,
        "name": "Lê Văn C",
        "subjects": ["Toán", "Lý"],
        "grades": [9, 10],
        "lat": 21.10,
        "lng": 105.90,
        "hourly_rate": 200_000,
        "profile_completed": 0.7,
        "is_verified_id": False,
        "is_verified_student_card": False,
        "total_classes": 4,
        "completed_classes": 2,
        "avg_rating": 3.9,
        "avg_response_time_sec": 7200,
        "policy_violations_count": 1,
        "free_slots": ["Tue_19-21"],
        "bio": "Tập trung luyện đề tư duy, dạy cả Toán và Vật Lý cho học sinh khá giỏi",
    },
    {
        "id": 4,
        "name": "Phạm Minh D",
        "subjects": ["Toán"],
        "grades": [7, 8, 9],
        "lat": 21.00,
        "lng": 105.75,
        "hourly_rate": 110_000,
        "profile_completed": 0.85,
        "is_verified_id": True,
        "is_verified_student_card": True,
        "total_classes": 10,
        "completed_classes": 8,
        "avg_rating": 4.5,
        "avg_response_time_sec": 900,
        "policy_violations_count": 0,
        "free_slots": ["Mon_19-21", "Fri_19-21"],
        "bio": "Kèm Toán nền tảng, củng cố hình học từ căn bản tới nâng cao",
    },
    {
        "id": 5,
        "name": "Hoàng Thu E",
        "subjects": ["Toán"],
        "grades": [8, 9, 10],
        "lat": 21.02,
        "lng": 105.78,
        "hourly_rate": 180_000,
        "profile_completed": 0.92,
        "is_verified_id": True,
        "is_verified_student_card": True,
        "total_classes": 15,
        "completed_classes": 14,
        "avg_rating": 4.9,
        "avg_response_time_sec": 400,
        "policy_violations_count": 0,
        "free_slots": ["Wed_19-21", "Sat_9-11"],
        "bio": "Chuyên Toán 9, luyện thi chuyên, chữa bài theo năng lực từng bạn",
    },
    {
        "id": 6,
        "name": "Đỗ Hải F",
        "subjects": ["Toán"],
        "grades": [9],
        "lat": 21.05,
        "lng": 105.83,
        "hourly_rate": 140_000,
        "profile_completed": 0.75,
        "is_verified_id": True,
        "is_verified_student_card": False,
        "total_classes": 5,
        "completed_classes": 4,
        "avg_rating": 4.0,
        "avg_response_time_sec": 2400,
        "policy_violations_count": 0,
        "free_slots": ["Mon_19-21", "Wed_20-22"],
        "bio": "Ôn thi vào 10, tập trung xử lý mất gốc và luyện đề hình học",
    },
]

request = {
    "subject": "Toán",
    "grade": 9,
    "lat": 21.02,
    "lng": 105.81,
    "budget_per_session": 150_000,
    "available_slots": ["Mon_19-21", "Wed_19-21"],
    "description": "Cần gia sư Toán lớp 9, ôn thi vào 10, con hơi mất gốc phần hình học",
}

In [5]:
# ==========================
# 2. TRUST SCORE (rule-based)
# ==========================

def calc_trust_score(tutor):
    total = max(tutor["total_classes"], 1)
    completion_rate = tutor["completed_classes"] / total
    avg_rating = tutor["avg_rating"] or 0
    rt = tutor["avg_response_time_sec"] or 9999
    violations = tutor["policy_violations_count"]

    score = 0.0

    # 1. H? s? (0-35)
    profile_score = tutor["profile_completed"] * 20
    if tutor["is_verified_id"]:
        profile_score += 8
    if tutor["is_verified_student_card"]:
        profile_score += 7
    profile_score = min(profile_score, 35)
    score += profile_score

    # 2. T? l? ho?n th?nh l?p (0-25)
    if completion_rate >= 0.9:
        completion_score = 25
    elif completion_rate >= 0.75:
        completion_score = 18
    elif completion_rate >= 0.5:
        completion_score = 10
    else:
        completion_score = 5
    score += completion_score

    # 3. ??nh gi? ph? huynh (0-25)
    rating_score = (avg_rating / 5.0) * 25
    score += rating_score

    # 4. Th?i gian ph?n h?i (0-10)
    if rt < 300:
        response_score = 10
    elif rt < 3600:
        response_score = 7
    elif rt < 6 * 3600:
        response_score = 4
    else:
        response_score = 2
    score += response_score

    # 5. Tu?n th? ch?nh s?ch (0-5, c? ph?t)
    if violations == 0:
        policy_score = 5
    elif violations == 1:
        policy_score = 2
    else:
        policy_score = 0
        score -= 10
    score += policy_score

    return max(0, min(score, 100))


In [6]:
# ==========================
# 3. SEMANTIC EMBEDDING (sentence-transformers)
# ==========================

model = SentenceTransformer("all-MiniLM-L6-v2")


def compute_semantic_scores(tutors: List[Dict], description: str):
    if not description.strip():
        return [0.5 for _ in tutors]
    req_emb = model.encode(description, normalize_embeddings=True)
    scores = []
    for t in tutors:
        bio = t.get("bio", "")
        if not bio:
            scores.append(0.5)
            continue
        emb = model.encode(bio, normalize_embeddings=True)
        sim = float(util.cos_sim(req_emb, emb))  # 0..1
        scores.append(sim)
    return scores


In [7]:
# ==========================
# 4. MATCHING ENGINE (rule + semantic)
# ==========================

def distance_km(lat1, lng1, lat2, lng2):
    R = 6371
    dlat = math.radians(lat2 - lat1)
    dlng = math.radians(lng2 - lng1)
    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlng / 2) ** 2
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def time_overlap_score(request_slots, tutor_slots):
    inter = set(request_slots) & set(tutor_slots)
    if not request_slots:
        return 0.0
    return len(inter) / len(request_slots)

def price_score(budget, rate):
    if rate <= budget:
        return 1.0
    diff = rate - budget
    if diff <= 20_000:
        return 0.8
    elif diff <= 50_000:
        return 0.5
    else:
        return 0.2

def distance_score_km(dist):
    if dist <= 2:
        return 1.0
    elif dist <= 5:
        return 0.8
    elif dist <= 10:
        return 0.5
    else:
        return 0.0

def calc_match_score(request, tutor, trust_score, semantic_score):
    subj_match = 1.0 if request["subject"] in tutor["subjects"] else 0.0
    grade_match = 1.0 if request["grade"] in tutor["grades"] else 0.0
    overlap = time_overlap_score(request["available_slots"], tutor["free_slots"])
    d = distance_km(request["lat"], request["lng"], tutor["lat"], tutor["lng"])
    dist_score = distance_score_km(d)
    p_score = price_score(request["budget_per_session"], tutor["hourly_rate"])
    trust_norm = trust_score / 100.0

    score = (
        0.25 * subj_match
        + 0.10 * grade_match
        + 0.15 * overlap
        + 0.10 * p_score
        + 0.10 * dist_score
        + 0.15 * trust_norm
        + 0.15 * semantic_score
    )
    return score


In [8]:
# ==========================
# 5. CHẠY DEMO
# ==========================

def run_demo():
    print("===== YÊU CẦU CỦA PHỤ HUYNH =====")
    print(request)
    print()

    print("===== TÍNH TRUST SCORE =====")
    for t in tutors:
        ts = calc_trust_score(t)
        t["trust_score"] = ts
        print(f"- {t['name']}: {ts:.2f}/100")
        print(f"   + Đã xác minh CCCD/CMND: {t.get('is_verified_id')}, Thẻ SV: {t.get('is_verified_student_card')}")
        print(f"   + Bio: {t.get('bio', '')}")
    print()

    semantic_scores = compute_semantic_scores(tutors, request["description"])

    print("===== KẾT QUẢ MATCHING (TOP) =====")
    results = []
    for t, sem in zip(tutors, semantic_scores):
        ms = calc_match_score(request, t, t["trust_score"], sem)
        results.append((ms, t, sem))

    results.sort(reverse=True, key=lambda x: x[0])

    for rank, (ms, t, sem) in enumerate(results, start=1):
        dist = distance_km(request['lat'], request['lng'], t['lat'], t['lng'])
        print(f"#{rank}: {t['name']}")
        print(f"   - Matching Score: {ms:.3f} (0-1)")
        print(f"   - Trust Score:    {t['trust_score']:.2f}/100")
        print(f"   - Semantic sim:   {sem:.3f}")
        print(f"   - Giá/buổi:       {t['hourly_rate']:,} VND")
        print(f"   - Khoảng cách:    {dist:.2f} km")
        print(f"   - Slots trống:    {set(request['available_slots']) & set(t['free_slots'])}")
        print(f"   - Xác minh:       CCCD/CMND: {t.get('is_verified_id')}, Thẻ SV: {t.get('is_verified_student_card')}")
        print(f"   - Bio:            {t.get('bio', '')}")
        print()

if __name__ == "__main__":
    run_demo()


===== YÊU CẦU CỦA PHỤ HUYNH =====
{'subject': 'Toán', 'grade': 9, 'lat': 21.02, 'lng': 105.81, 'budget_per_session': 150000, 'available_slots': ['Mon_19-21', 'Wed_19-21'], 'description': 'Cần gia sư Toán lớp 9, ôn thi vào 10, con hơi mất gốc phần hình học'}

===== TÍNH TRUST SCORE =====
- Nguyễn Văn A: 94.00/100
   + Đã xác minh CCCD/CMND: True, Thẻ SV: True
   + Bio: Gia sư Toán 9, luyện thi vào 10, thế mạnh hình học và tư duy giải nhanh
- Trần Thị B: 75.50/100
   + Đã xác minh CCCD/CMND: True, Thẻ SV: False
   + Bio: Dạy Toán cơ bản đến nâng cao, ôn nền tảng cho học sinh mất gốc
- Lê Văn C: 49.50/100
   + Đã xác minh CCCD/CMND: False, Thẻ SV: False
   + Bio: Tập trung luyện đề tư duy, dạy cả Toán và Vật Lý cho học sinh khá giỏi
- Phạm Minh D: 84.50/100
   + Đã xác minh CCCD/CMND: True, Thẻ SV: True
   + Bio: Kèm Toán nền tảng, củng cố hình học từ căn bản tới nâng cao
- Hoàng Thu E: 94.90/100
   + Đã xác minh CCCD/CMND: True, Thẻ SV: True
   + Bio: Chuyên Toán 9, luyện thi chuyên, chữ